In [1]:
import pandas as pd

In [2]:
res_df1 = pd.read_csv("synth_results_many.csv")

In [3]:
res_df = pd.read_csv("synth_results_w_recon.csv")

In [4]:
res_df.sort_values(['seedmethod', 'bcv', 'dropout']).to_csv('synth_results_w_recon_sorted.csv')

In [9]:
res_df

,seedmethod,bcv,dropout,demap,accuracy,recon score
0,"groups,46",0.2,0.7,0.746910,0.750408,0.624526
1,"groups,46",0.2,0.7,0.718545,0.748234,-0.009220
2,"paths,44",0.4,0.5,0.588830,0.724110,-0.029067
3,"paths,44",0.4,0.5,0.579844,0.730167,0.402261
4,"groups,43",0.6,0.2,0.583970,0.715159,-0.013807
...,...,...,...,...,...,...
175,"groups,42",0.2,0.5,0.777820,0.747658,0.678843
176,"paths,44",0.4,0.7,0.569473,0.727922,0.346185
177,"groups,43",0.6,0.7,0.545900,0.701400,-0.013308
178,"groups,43",0.6,0.7,0.468422,0.692136,0.270351


In [5]:
# res_df1.sort_values(['dataset', 'bcv', 'dropout']).to_csv('syth_results_many_sorted.csv')

In [6]:
res_df1

,dataset,bcv,dropout,recon_weights,acc_our_train,acc_our_test,demap_phate_train,demap_our_train,demap_our_test
0,"groups,46",0.2,0.7,"[0.9, 0.1, 0]",0.790894,0.690511,0.564295,0.537111,0.746910
1,"groups,46",0.2,0.7,"[1, 0, 0]",0.631455,0.585316,0.564295,0.559419,0.718545
2,"paths,44",0.4,0.5,"[1, 0, 0]",0.840235,0.628667,0.509145,0.517595,0.588830
3,"paths,44",0.4,0.5,"[0.9, 0.1, 0]",0.788573,0.628577,0.509145,0.516378,0.579844
4,"groups,43",0.6,0.2,"[1, 0, 0]",0.839348,0.630345,0.418876,0.423508,0.583970
...,...,...,...,...,...,...,...,...,...
175,"groups,42",0.2,0.5,"[0.9, 0.1, 0]",0.747457,0.677960,0.822904,0.822629,0.777820
176,"paths,44",0.4,0.7,"[0.9, 0.1, 0]",0.776023,0.608725,0.486966,0.492818,0.569473
177,"groups,43",0.6,0.7,"[1, 0, 0]",0.845257,0.569284,0.424783,0.424342,0.545900
178,"groups,43",0.6,0.7,"[0.9, 0.1, 0]",0.666802,0.547300,0.424783,0.425989,0.468422


In [11]:
res_df_merge = pd.concat([res_df, res_df1['recon_weights']], axis=1)

In [13]:
res_df_merge['recon_weights'] = res_df_merge['recon_weights'].apply(lambda x: (x == '[0.9, 0.1, 0]'))
res_df_merge.rename(columns={'recon_weights': 'Reconstruction Loss'}, inplace=True)

In [16]:
res_df_merge.sort_values(['seedmethod', 'bcv', 'Reconstruction Loss'], inplace=True)

In [17]:
res_df_merge.columns

Index(['seedmethod', 'bcv', 'dropout', 'demap', 'accuracy', 'recon score',
       'Reconstruction Loss'],
      dtype='object')

In [18]:
res_df_merge.columns
cols = ['seedmethod', 'bcv', 'dropout', 'Reconstruction Loss', 'demap', 'accuracy', 'recon score']
res_df_merge = res_df_merge[cols]

In [19]:
import pandas as pd

mean_df = res_df_merge.iloc[:,1:].groupby(['bcv', 'dropout', 'Reconstruction Loss']).mean().round(3)
std_df = res_df_merge.iloc[:,1:].groupby(['bcv', 'dropout', 'Reconstruction Loss']).std().round(3)

# Merge the mean and std dataframes
combined_df = mean_df.merge(std_df, left_index=True, right_index=True, suffixes=('_mean', '_std'))

# Format the results into a single column for each feature
for col in mean_df.columns:  # Iterate directly through the columns
    combined_df[col] = combined_df[col+'_mean'].astype(str) + "(±" + combined_df[col+'_std'].astype(str) + ")"
    # Since we're directly modifying the combined_df, no need to drop the columns in this loop

# Drop the original mean and std columns after processing
combined_df = combined_df.loc[:, ~combined_df.columns.str.contains('_mean|_std')]

# combined_df will now have the desired format
print(combined_df)

                                         demap       accuracy     recon score
bcv dropout Reconstruction Loss                                              
0.2 0.2     False                 0.706(±0.09)   0.772(±0.03)  -0.006(±0.029)
            True                 0.705(±0.087)  0.775(±0.028)   0.672(±0.066)
    0.5     False                0.688(±0.095)   0.774(±0.03)    0.006(±0.03)
            True                 0.706(±0.089)  0.776(±0.023)   0.645(±0.066)
    0.7     False                0.675(±0.094)  0.766(±0.031)   0.002(±0.022)
            True                 0.678(±0.096)  0.768(±0.029)   0.594(±0.078)
0.4 0.2     False                0.622(±0.125)  0.748(±0.033)    -0.0(±0.022)
            True                 0.626(±0.128)   0.753(±0.03)   0.469(±0.093)
    0.5     False                0.613(±0.128)  0.752(±0.036)   -0.003(±0.02)
            True                 0.608(±0.126)   0.75(±0.033)    0.447(±0.09)
    0.7     False                 0.59(±0.138)  0.738(±0.026)  -

In [20]:
combined_df.to_csv('synth_res_final.csv', index=False)